In [12]:
import cv2
import numpy as np
import time
import base64
import socketio

In [15]:
#카메라 열기 
cap = cv2.VideoCapture(0)

#카메라 Width Height 설정
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

#인코딩용 전송 파라미터 설정
encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 90]

#소켓통신을 위한 준비
isSocket = True

#소켓 호출 에러시 처리하기 위한 부분
try:
    sio = socketio.Client()
    sio.connect('http://localhost:746')
except:
    isSocket = False

if isSocket == True:
    while True:
        time.sleep(0.1)
        #카메라로 부터 사진 한장 읽기 
        ret, frame = cap.read()

        frame = cv2.flip(frame, 1)
        
        scResult, scFrame = cv2.imencode('.jpg', frame, encode_param)        
        scData = base64.b64encode(scFrame)

        #소켓으로 이미지 전송
        sio.emit('streaming', scData)
        
        cv2.imshow('Person Detect Client', frame)

        if cv2.waitKey(1) == 13:
            break

    sio.disconnect()
    cap.release()
    cv2.destroyAllWindows()    
else:
    print("소켓접속 불량")